In [1]:
# importing image object from PIL
import numpy as np
import math
from PIL import Image, ImageDraw

""" 
# This is for the google colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/
"""

# file read
def fileRead(file_name):
    """ Read find and return the values stored in the file. 
    @arguemnt(file_name): file_name. File should be stored at the same directory
    @return: start (X and Y coordinates),  end (X and Y coor, goal state)
             n_rect (Number of rectangles), rect_list (list of rect coordiates, 
             ex. [((0, 0), (5, 0), (5, 5), (0, 5)), ((6, 2), (8, 2), (8, 25), (6, 25))
    """    
    f = open(file_name, 'r')
    # Start point (data structre: tuple)
    start = f.readline().strip().split(" ")
    start = (int(start[0]),int(start[1]))
    
    # End point (data structre: tuple)
    end = f.readline().strip().split(" ")
    end = (int(end[0]),int(end[1]))
    
    # Number of rectangles 
    n_rect = int(f.readline().strip())
    
    # List of rectangles
    rect_list = []
    for i in range(n_rect):
        rect = f.readline().strip().split(" ")
        p1 = (int(rect[0]),int(rect[1]))
        p2 = (int(rect[2]),int(rect[3]))
        p3 = (int(rect[4]),int(rect[5]))
        p4 = (int(rect[6]),int(rect[7]))
        rect = (p1,p2,p3,p4)
        rect_list.append(rect)
        
    f.close()
    return start, end, n_rect, rect_list

# Draw
def draw(start, end, n_rect, rect_list):
    """ Draw rectangles and their coordinates. Also, draw start and end states
    @arguemnt(start, end, n_rect, rect_list): start coordinate, goal coordinate, 
        number of rectangles, and list of rect coordiates
    """
    # To convert coords to pixel points, we need magnify. Otherwise, images look too small. 
    MAGNIFY = 20        
    MARGIN = MAGNIFY*2  # Margin for boundary padding
    
    #rect_list
    w, h = np.amax(rect_list)*MAGNIFY+MARGIN*2, np.amax(rect_list)*MAGNIFY+MARGIN*2

    # creating new Image object
    img = Image.new("RGB", (w, h))

    # draw rectangle image
    for rect in rect_list:
        image = ImageDraw.Draw(img) 
        # Draw rectangles 
        rect_coor = np.array(rect)*MAGNIFY+MARGIN        
        rect_coor = tuple([tuple(vertex) for vertex in rect_coor])    
        image.polygon(rect_coor, fill ="grey", outline ="white") 
        # Draw text (coordinates of vertices of each rectangles)
        for vertex in rect:
            vertex_coor = "("+str(vertex[0])+","+str(vertex[1])+")"
            image.text((vertex[0]*MAGNIFY+MARGIN+3, vertex[1]*MAGNIFY+MARGIN-10), vertex_coor, fill="yellow")

    # draw start and end points
    start = np.array(start*2)*MAGNIFY+MARGIN
    start = (start[0]-2, start[1]-2, start[2]+2, start[3]+2)
    image.ellipse(start, fill="red")
    image.text((start[0]+2, start[1]-16),"START", fill="red")

    end = np.array(end*2)*MAGNIFY+MARGIN
    end = (end[0]-2, end[1]-2, end[2]+2, end[3]+2)
    image.ellipse(end, fill="red")
    image.text((end[0]+2, end[1]-16),"END", fill="red")
        
    img.show() # For Jupyter Notebook
    #display(img) # For Google Colab



In [2]:
def get_line_intersection(p0_x, p0_y, p1_x, p1_y, p2_x, p2_y, p3_x, p3_y):
    """ Check two lines are intersect
    @arguemnt(Each points of end of lines): p0,p1-->end points of line1, p2,p3-->end points of line2 
    @return: line is intersect return True, otherwise return False 
    """    
    #print(p0_x, p0_y, p1_x, p1_y, p2_x, p2_y, p3_x, p3_y)
    s1_x = p1_x - p0_x     
    s1_y = p1_y - p0_y
    s2_x = p3_x - p2_x     
    s2_y = p3_y - p2_y
    
    # Check parallel lines
    if s1_x != 0 and s2_x != 0: # Check for divide by zero 
        if (s1_y/s1_x == s2_y/s2_x): #Parallel(same slope) means no collision
            return False # No Collision
    else: 
        if (s1_y==s2_y or s1_x==s2_x):
            return False

    # Check end of line is collision 
    if (p0_x==p2_x and p0_y==p2_y) or (p0_x==p3_x and p0_y==p3_y) or \
        (p1_x==p2_x and p1_y==p2_y) or (p1_x==p3_x and p1_y==p3_y):
        return False # No Collision. Only end of lines meet

    # Not parallel lines 
    s = (-s1_y * (p0_x - p2_x) + s1_x * (p0_y - p2_y)) / (-s2_x * s1_y + s1_x * s2_y)
    t = ( s2_x * (p0_y - p2_y) - s2_y * (p0_x - p2_x)) / (-s2_x * s1_y + s1_x * s2_y)

    if(s >= 0 and s <= 1 and t >= 0 and t <= 1):
        # Collision detected
        i_x = p0_x + (t * s1_x);
        i_y = p0_y + (t * s1_y);
        return True # Return True when two lines are collied

    return False #No Collision

def line_rect_collision(s1, s2, rect_list):
    """ Check if the line connecting s1 point to s2 point intersect rectangles. (Illegal step)
        We can check selecting s2 is the correct move. 
    @arguemnt: s1 and s2 are coordinates connecting line. rect_list are coordinates of vertices of all rectangles.
    @return: True if line connecting s1 and s2 intersects one of rectangles. (Illegal step)
            False if line connecting s1 and s2 does not intersect any rectangle. (Legal step. We can expand s2.)
    """    
    for rect in rect_list:
        # Line intersection test between line from current state to candidate state and edges of rectangle
        if (get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[0][0], rect[0][1], rect[1][0], rect[1][1]) or \
            get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[1][0], rect[1][1], rect[2][0], rect[2][1]) or \
            get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[2][0], rect[2][1], rect[3][0], rect[3][1]) or \
            get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[3][0], rect[3][1], rect[0][0], rect[0][1])):
            return True # Collision detected

        # If line hits diagnal line in side of rectangle --> Illegal. The line cross inside of rectangle. 
        if (get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[0][0], rect[0][1], rect[2][0], rect[2][1]) or \
            get_line_intersection(s1.xy[0], s1.xy[1], s2.xy[0], s2.xy[1], rect[1][0], rect[1][1], rect[3][0], rect[3][1])):
            return True

    return False #No Collision between line and all rectangles   

In [3]:
from numpy.core.fromnumeric import put
from queue import PriorityQueue

 # Class for the each state
class State(object):
    def __init__(self, xy):
        # Argument: Coordinate of points
        self.xy = xy    # The coordinates of a point
        self.g = None   # g-value cost of the path from the initial state to this state
        self.h = None   # h-value that estimates the cost from this state to the goal
        self.f = None   # f-value that is the sum of these two
        self.parent = None  #the parent state
        # Note) a list of the successors of this state seems unnecessary

# Class for PathFinder problem 
class PathFinder(object):
    def __init__(self, start, end, n_rect, rect_list):
        #Input: self(coords of start), end(coords of end), n_rect(# of rect), rect_list(coords of vertices)
        self.start = State(start)
        self.end = State(end)
        self.n_rect = n_rect
    
    def startState(self):
        # Initialize the start node, update g,h,f for start node
        self.start.g = 0
        self.start.h = self.getHeuristic(self.start)
        self.start.f = self.start.g + self.start.h
        return

    def isEnd(self, state):
        # Check for end state. It reaches to the goal. 
        if self.end.xy == state.xy:
            # print
            print("============ Solution =========")
            print('{:<10}{:<10}'.format("Point", "Cumulative Cost"))
            prt = []
            while state is not None:
                prt.insert(0,(state.xy, state.g))  
                state = state.parent
            for s in prt:
                print('{:<10}{:<10}'.format(str(s[0]), str(s[1])))
            print("===============================")
            return True
        else:
            return False

    def getG(self, state):
        # Distance from current note to parent node
        p0_x, p0_y = state.parent.xy
        p1_x, p1_y = state.xy
        dist = ((p0_x-p1_x)**2 + (p0_y-p1_y)**2)**(1/2)
        # g = Distance so far from the start note = dist + parent.g
        return dist + state.parent.g

    def getHeuristic(self, state):
        # Heuristic value: straight line distance from a given state to end state
        return ((self.end.xy[0]-state.xy[0])**2 + (self.end.xy[1]-state.xy[1])**2)**(1/2)

    def succAction(self, state):
        """ Return the successive states from current state 
        @arguemnt(state): current state 
        @return(result): List of children (possible new states)
        """
        result = []
        # Check possible roots. Cannot go through rectangle 
        # TO DO: Need to modify to queue
        for rect in rect_list:
            for vertex in rect:
                if state.xy != vertex: 
                    vertex = State(vertex)
                    if not line_rect_collision(state, vertex, rect_list): 
                        result.append(vertex)
        return result
                
def A_star(problem):
    # ============================= Initializatoin ============================= 
    open = PriorityQueue()
    closed = [] # CLOSED <- empty list
    # Initialize the start node, update g,h,f for start node
    problem.startState()
    # OPEN <- start node
    open.put((problem.start.f, problem.start)) 
    # ============================ Repeat until goal ===========================
    while True:
        if open.empty(): # if OPEN is empty, fail
            print("Queue is empty")
            return False
        f, bestNode = open.get() # BESTNODE <- node on OPEN with lowest f
        if problem.isEnd(bestNode): # if BESTNODE is a goal, exit and succeed
            return True

        #remove BESTNODE from OPEN and add it to CLOSED
        closed.append(bestNode)

        #generate successors of BESTNODE
        for s in problem.succAction(bestNode):
            #for each successor s do
            # 1. set its parent field
            s.parent = bestNode
            # 2. compute g(s)
            s.g = problem.getG(s) 
            
            # Check s is in OPEN and CLOSED
            s_in_open = find_State_in_Queue(s, open)
            s_in_closed = find_State_in_List(s, closed)
            
            # 3. if there is a node OLD on OPEN with the same state info as s
            if (s_in_open):
                # if g(s) < g(OLD), update OLD and throw out s
                update_State_in_Open(s,open) 
            # 4. if (s is not on OPEN and there is a node OLD on CLOSED with the same state info as s
            elif ((not s_in_open) and s_in_closed):
                update_move_State_in_Closed(s,open,closed)
            # 5. If s was not on OPEN or CLOSED
            elif (not s_in_open) and (not s_in_closed):
                # update parent
                s.parent = bestNode
                # calculate g(s), h(s), f(s) of s
                s.h = problem.getHeuristic(s)
                s.f = s.g + s.h
                # add s to OPEN
                open.put((s.f, s)) 

def find_State_in_Queue(s,q):
    """ Find the state in queue
    @ Input: State, queue(OPEN) 
    @ Return: True/ False"""
    for i in range(q.qsize()):
        if (s.xy == q.queue[i][1].xy):
            return True
    return False 

def find_State_in_List(s, lst):
    """ Find the state in List
    @ Input: State, list(CLOSED) """
    for l in lst:
        if l.xy == s.xy:
            return True
    return False
  
def update_State_in_Open(s,q):
    """ if g(s) < g(OLD), update OLD and throw out s
    @ Input: State, queue(OPEN)"""
    for i in range(q.qsize()):
        if (s.xy == q.queue[i][1].xy):
            if s.g < q.queue[i][1].g: 
                q.queue[i][1].f = s.f
                q.queue[i][1].f = s.g
                q.queue[i][1].parent = s.parent

def update_move_State_in_Closed(s,q,lst):
    """if g(s) < g(OLD), update OLD, remove it from CLOSED and put it on OPEN, throw out s
    @ Input: State, queue(OPEN), list(CLOSED)"""
    for l in lst:
        if l.xy == s.xy:
            if s.g < l.g: 
                q.put((s.f, s)) 
                lst.remove(l)
                print("g(s) < g(OLD), OLD in CLOSED")


In [4]:
file_list = []
file_list.append("simple.txt")
file_list.append("difficult.txt")

for file_name in file_list:
    print("\n*************************** {} ***************************".format(file_name))
    start, end, n_rect, rect_list = fileRead(file_name)
    print(" start: {}\n end: {}\n n_rect: {}\n rect_list: {}\n ".format(start, end, n_rect, rect_list))
    draw(start, end, n_rect, rect_list) # Draw images

    pathFinder = PathFinder(start, end, n_rect, rect_list) 
    if (A_star(pathFinder)):
        print("Find the solution\n\n")
    else:
        print("Find no solution\n\n")


*************************** EE562_hw2_simple.txt ***************************
 start: (0, 0)
 end: (9, 6)
 n_rect: 2
 rect_list: [((0, 0), (4, 0), (4, 4), (0, 4)), ((7, 4), (9, 6), (4, 10), (2, 8))]
 
============ Solution =========
Point     Cumulative Cost
(0, 0)    0         
(4, 0)    4.0       
(9, 6)    11.810249675906654
Find the solution



*************************** EE562_hw2_difficult.txt ***************************
 start: (0, 0)
 end: (25, 35)
 n_rect: 10
 rect_list: [((0, 0), (5, 0), (5, 5), (0, 5)), ((6, 2), (8, 2), (8, 25), (6, 25)), ((10, 10), (12, 10), (12, 12), (10, 12)), ((15, 0), (20, 5), (15, 10), (10, 5)), ((10, 15), (15, 13), (17, 18), (12, 20)), ((24, 4), (26, 6), (21, 11), (19, 9)), ((28, 7), (30, 9), (22, 17), (20, 15)), ((26, 19), (31, 24), (30, 25), (25, 20)), ((20, 20), (25, 25), (15, 35), (10, 30)), ((22, 33), (25, 33), (25, 35), (22, 35))]
 
============ Solution =========
Point     Cumulative Cost
(0, 0)    0         
(5, 0)    5.0       
(8, 2)    8.60